<h3> Work From Calorie Calulator</h3>

Reflecting on my year in an AP Computer Science Principles (AP CSP) class, there are several key takeaways and insights that come to mind. This course is designed to introduce students to the world of computer science, programming, and computational thinking, and it provides a foundation for understanding the broader impact of technology on society. Here are some of my personal reflections on the experience:

Broadening My Perspective: AP CSP has exposed me to a diverse range of computing concepts, from algorithms and data to the ethical and social implications of technology. It has encouraged me to think beyond coding and to understand how technology affects society.

Developing Problem-Solving Skills: One of the primary objectives of AP CSP is to develop problem-solving skills. I've learned how to approach complex problems, break them down into manageable parts, and devise solutions through programming. These skills are not only valuable in computer science but also in various aspects of life.

Coding and Abstraction: I've gained a fundamental understanding of coding and abstraction, realizing that complex problems can be broken down into simpler steps and coded in a way that a computer can execute. This abstraction is a key concept in computer science and extends to many areas of life.

Collaboration: The course often includes collaborative projects, which have taught me how to work effectively in teams. Collaboration is a crucial skill, not only in computer science but also in most professional settings.

Creative Expression: Computer science is not just about numbers and logic; it's also a form of creative expression. In AP CSP, I've discovered how to create programs and interactive applications that reflect my ideas and creativity.

Ethical Considerations: AP CSP delves into the ethical and societal impacts of technology. It has prompted me to consider issues like privacy, security, and the responsible use of data. These discussions are vital in a world increasingly shaped by technology.

Real-World Relevance: This course has demonstrated how computer science is relevant to real-world problems. I've learned how computational thinking can be applied to various fields, from healthcare and finance to environmental science.

Preparation for Further Studies: For me, as someone who plans to pursue computer science in college, AP CSP has provided an excellent foundation. It has introduced me to the basics of programming, data analysis, and more, which I believe will make the transition to higher-level courses smoother.

Overcoming Challenges: Computer science can be challenging, but through this course, I've learned how to persevere, debug code, and overcome obstacles. These problem-solving skills are transferable to other areas of life.

Life Skills: Beyond the technical aspects, AP CSP has equipped me with critical life skills, such as adaptability, logical thinking, and a growth mindset. These skills are invaluable in a world that is constantly evolving.

In conclusion, my experience in an AP CSP class has offered me a rich and multifaceted learning journey. It has not only equipped me with essential computer science knowledge but has also fostered a deeper understanding of how technology impacts the world. The course has instilled problem-solving skills, ethical awareness, and the ability to collaborate effectively, making it a valuable educational experience for me as I pursue my interests in computer science and seek to develop important life skills.

In [ ]:
import threading

# import "packages" from flask
from flask import render_template  # import render_template from "public" flask libraries
from flask_sqlalchemy import SQLAlchemy

# import "packages" from "this" project
from __init__ import app,db  # Definitions initialization
from model.jokes import initJokes
from model.users import initUsers
from model.players import initPlayers
from model.calcalcs import initCalories

# setup APIs
from api.covid import covid_api # Blueprint import api definition
from api.joke import joke_api # Blueprint import api definition
from api.user import user_api # Blueprint import api definition
from api.player import player_api
from api.calcalc import calcalc_api

# setup App pages
from projects.projects import app_projects # Blueprint directory import projects definition


# Initialize the SQLAlchemy object to work with the Flask app instance
db.init_app(app)

# register URIs
app.register_blueprint(calcalc_api)
app.register_blueprint(joke_api) # register api routes
app.register_blueprint(covid_api) # register api routes
app.register_blueprint(user_api) # register api routes
app.register_blueprint(player_api)
app.register_blueprint(app_projects) # register app pages

@app.errorhandler(404)  # catch for URL not found
def page_not_found(e):
    # note that we set the 404 status explicitly
    return render_template('404.html'), 404

@app.route('/')  # connects default URL to index() function
def index():
    return render_template("index.html")

@app.route('/table/')  # connects /stub/ URL to stub() function
def table():
    return render_template("table.html")



@app.before_first_request
def activate_job():  # activate these items 
    initJokes()
    initUsers()
    initPlayers()

# this runs the application on the development server
if __name__ == "__main__":
    # change name for testing
    from flask_cors import CORS
    cors = CORS(app)
    app.run(debug=True, host="0.0.0.0", port="8169")


In [ ]:
class CalCalcAPI(Resource):
    def post(self):
        try:
            data = request.get_json()

            # Ensure that all required fields are present in the request JSON
            required_fields = ['age', 'weight', 'height', 'activity', 'gender', 'username']
            for field in required_fields:
                if field not in data:
                    return {'error': f"Missing field: {field}"}, 400

            # Calculate calorie maintenance (you can reuse your client-side logic here)
            calorie_maintenance = calculateCalories(data)

            # Calculate the meal plan based on calorie maintenance
            mealPlan = ''
            mealPlanNumber = 0
            if calorie_maintenance < 1000:
                mealPlan = 'Meal Plan 1'
            else:
                mealPlanNumber = (calorie_maintenance - 1000) // 250 + 1
                mealPlan = f'Meal Plan {mealPlanNumber}'
